In [2]:
import pandas as pd
import torch
import torch.nn as nn 
import torch.optim as optim
import random
import pandas

In [3]:
#data_generator
matrix_T = torch.tensor([[0, 1], [1, 0]], dtype=torch.float64)
matrix_U = torch.tensor([[1, -1], [0, 1]], dtype=torch.float64)

inverse_T = torch.inverse(matrix_T)
inverse_U = torch.inverse(matrix_U)

max_moves = 30  #how many inverse operations on the final destination
goal_position = torch.tensor([[1], [0]], dtype=torch.float64)


def data_generator(num_of_datapoints):
    data = torch.empty((num_of_datapoints, 3), dtype=torch.float64)
    for i in range(num_of_datapoints):
        start = goal_position
        moves = random.randint(1, max_moves)
        coin = 3
        for q in range(moves):
            coin = random.randint(0, 1)
            if coin == 1:
                #  print(start, inverse_U, "=")
                start = torch.matmul(inverse_U, start)
            #  print(start)
            else:
                #  print(start, inverse_T, "=")
                start = torch.matmul(inverse_T, start)
                if q != moves - 1:
                    start = torch.matmul(inverse_U, start)
            # print(start)
        # print("done")
        data[i, 0] = start[0]
        data[i, 1] = start[1]
        data[i, 2] = coin
    return data

#https://stackoverflow.com/questions/36158058/torch-save-tensor-to-csv-file#:~:text=For%20simple%20tables%2C%20you%20can,then%20to%20a%20Pandas%20dataframe.&text=You%20can%20first%20convert%20the,table%20as%20a%20csv%20file.
data = data_generator(20000)
data_table = data.numpy()
df = pandas.DataFrame(data_table)
df.to_csv("generated_pairs.csv", index = False) 


In [4]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
#https://stackoverflow.com/questions/12336234/read-csv-file-to-numpy-array-first-row-as-strings-rest-as-float
#https://www.w3schools.com/python/numpy/numpy_array_join.asp#:~:text=Joining%20Arrays%20Using%20Stack%20Functions&text=We%20can%20concatenate%20two%201,it%20is%20taken%20as%200.

#make tensors from csv file

#from cs544 notes 
data = pd.read_csv("generated_pairs.csv", names=["0","1","2"], delimiter=",")
dataset = torch.utils.data.TensorDataset(
    torch.tensor(data.loc[:, "0":"1"].values), # input 
    torch.tensor(data.loc[:, ["2"]].values) #output 
)
X, Y = dataset[:] #label input and output 


In [5]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
#making the model 
model = nn.Sequential(
    nn.Linear(2, 128), #first layer 128
    nn.ReLU(),
    nn.Linear(128,64), #second layer 64
    nn.ReLU(),
    nn.Linear(64, 16), #third layer 16
    nn.ReLU(),
    nn.Linear(16,1), #singular output 
    nn.Sigmoid()
)
#print(model)

In [6]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
#prep for training 
loss_function = nn.BCELoss() #should this be our loss function too? 
optimizer = optim.Adam(model.parameters(), lr = 0.001) #chose Adam and lr from site 

#split training and testing data 
train, test = torch.utils.data.random_split(dataset, [0.75, 0.25]) #from cs544 class notes 
X_train, Y_train = train[:]
X_train = torch.Tensor.numpy(X_train)
Y_train = torch.Tensor.numpy(Y_train)


In [7]:
batch_s = 100 #batch size 
epoch_s = 100 #epoch size 
dl = torch.utils.data.DataLoader(train, batch_size = batch_s, shuffle = True) #from cs544 notes 
dl_test = torch.utils.data.DataLoader(test, batch_size = batch_s, shuffle = False)

In [8]:
#https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
for epoch in range(epoch_s):
    total_loss = 0.0
    for i in range(0, len(X_train), batch_s):
        optimizer.zero_grad()
        X_batch = X_train[i:i+batch_s]
        # print(X_batch)
        y_pred = model(torch.Tensor(X_batch))
        Y_batch = Y_train[i:i+batch_s]
        # print(Y_batch)
        loss = loss_function(y_pred, torch.Tensor(Y_batch))
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Finished epoch {epoch}, latest loss {total_loss / len(dl)}')

Finished epoch 0, latest loss 0.24666918212831612
Finished epoch 1, latest loss 0.00949612499865401
Finished epoch 2, latest loss 0.003164393580418438
Finished epoch 3, latest loss 0.0021115233899064866
Finished epoch 4, latest loss 0.0017870842619484525
Finished epoch 5, latest loss 0.001631502027437168
Finished epoch 6, latest loss 0.001530113690276399
Finished epoch 7, latest loss 0.0014513107695611385
Finished epoch 8, latest loss 0.0013872486418625802
Finished epoch 9, latest loss 0.0013318697681174642
Finished epoch 10, latest loss 0.0012824606302629699
Finished epoch 11, latest loss 0.0012368533147586855
Finished epoch 12, latest loss 0.0011965680175446613
Finished epoch 13, latest loss 0.001163904855046298
Finished epoch 14, latest loss 0.0011373563162569613
Finished epoch 15, latest loss 0.0011143861196229554
Finished epoch 16, latest loss 0.0010944978657006753
Finished epoch 17, latest loss 0.0010765786845580963
Finished epoch 18, latest loss 0.0010610906238492659
Finished ep

In [16]:
X_test, Y_test = test[:]

X_test = torch.Tensor.numpy(X_test)
Y_test = torch.Tensor.numpy(Y_test)


In [20]:
# sum the differences between test output and the actual output
# Thankfully this is very small, which is good, so our model is good. 
# There are 10000 elements here in X_test (I think), so a sum of 0.57 is very good and very small
sum(model(torch.Tensor(X_test)) - torch.Tensor(Y_test))

tensor([0.5718], grad_fn=<AddBackward0>)